In [1]:
import os
import re
import openai
import pandas as pd
from llama_index.llms import AzureOpenAI
from llama_index.embeddings import OpenAIEmbedding
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index import download_loader
from llama_index import StorageContext, load_index_from_storage
from llama_index import set_global_service_context

In [2]:
openai.api_type = "azure"
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_version = os.environ['OPENAI_API_VERSION']
openai.api_key = os.environ['OPENAI_API_KEY']

llm = AzureOpenAI(
    model="gpt-35-turbo",
    engine="gpt-35-turbo",
    api_type="azure",
    api_key=os.environ['OPENAI_API_KEY'],
    api_base=os.environ['OPENAI_API_BASE'],
    api_version=os.environ['OPENAI_API_VERSION']
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = OpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_type="azure",
    api_key=os.environ['OPENAI_API_KEY'],
    api_base=os.environ['OPENAI_API_BASE'],
    api_version=os.environ['OPENAI_API_VERSION']
)

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)
set_global_service_context(service_context)

In [3]:
PDFReader = download_loader("PDFReader")
loader = PDFReader()

In [4]:
# Load the document
documents = loader.load_data(file="2401.04088.pdf")

# Create the vectorstore
index = VectorStoreIndex.from_documents(documents)

# Create the Query Engine
query_engine = index.as_query_engine()

In [5]:
def ask(question, context="", system=""):

    # Set default System Message
    if system == "":
        system = """You are an expert in Artificial Intelligence Research Papers. 
Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
"""

    # Prepend context if used
    if context != "":
        question = "Use the following context to answer the users question:\n```\n" + context + "\n```\n\n" + question

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages = [{"role":"system","content":system},{"role":"user","content":question}],
        temperature=0.0,
        max_tokens=500,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)
        
    return response['choices'][0]['message']['content']

In [6]:
def extract_section(documents, section_name, debug=False):
    section_page = ""
    section_text = ""

    for idx, page in enumerate(documents):
        if section_text == "" and section_name in page.text.lower():
            if debug: print(idx)

            context = page.text
            if idx < len(documents)-2:
                context += "\n" + documents[idx+1].text
                context += "\n" + documents[idx+2].text

            answer = ask(f"Does the above have the section called '{section_name}' or similar, and does it, in detail, explain the {section_name}?", context)
            if answer.startswith("Yes"):
                answer = ask(f"\n-----\nWhat is the {section_name} in the document? Return everything in this section, up to the next heading. Do not interpret it, give me the verbatim text.", context)
                if debug: print(answer + "\n----------")
                section_page = idx + 1
                section_text = answer
                if debug: print(section_page, section_text, validate)

    return section_text, section_page

In [7]:
sections = {}

sections["authors"] = (ask("Who are the authors mentioned before the abstract", documents[0].text), 1)
sections["authors"]

('The authors mentioned before the abstract are:\n1. Albert Q. Jiang\n2. Alexandre Sablayrolles\n3. Antoine Roux\n4. Arthur Mensch\n5. Blanche Savary\n6. Chris Bamford\n7. Devendra Singh Chaplot\n8. Diego de las Casas\n9. Emma Bou Hanna\n10. Florian Bressand\n11. Gianna Lengyel\n12. Guillaume Bour\n13. Guillaume Lample\n14. Lélio Renard Lavaud\n15. Lucile Saulnier\n16. Marie-Anne Lachaux\n17. Pierre Stock\n18. Sandeep Subramanian\n19. Sophia Yang\n20. Szymon Antoniak\n21. Teven Le Scao\n22. Théophile Gervet\n23. Thibaut Lavril\n24. Thomas Wang\n25. Timothée Lacroix\n26. William El Sayed',
 1)

In [8]:
sections["abstract"] = extract_section(documents, "abstract")
sections["abstract"]

('We introduce Mixtral 8x7B, a Sparse Mixture of Experts (SMoE) language model. Mixtral has the same architecture as Mistral 7B, with the difference that each layer is composed of 8 feedforward blocks (i.e. experts). For every token, at each layer, a router network selects two experts to process the current state and combine their outputs. Even though each token only sees two experts, the selected experts can be different at each timestep. As a result, each token has access to 47B parameters, but only uses 13B active parameters during inference. Mixtral was trained with a context size of 32k tokens and it outperforms or matches Llama 2 70B and GPT-3.5 across all evaluated benchmarks. In particular, Mixtral vastly outperforms Llama 2 70B on mathematics, code generation, and multilingual benchmarks. We also provide a model fine-tuned to follow instructions, Mixtral 8x7B – Instruct, that surpasses GPT-3.5 Turbo, Claude-2.1, Gemini Pro, and Llama 2 70B – chat model on human benchmarks. Bot

In [16]:
%%time

query = 'What licenses are mentioned?'
print(query)
answer = query_engine.query(query)
print(answer.response)
find = re.findall(r"'page_label': '[^']*'", str(answer.metadata))
page_num = eval("{" + find[0] + "}")["page_label"]

What licenses are mentioned?
The licenses mentioned in the context information are not provided.
CPU times: total: 15.6 ms
Wall time: 1.79 s


In [17]:
%%time

ask(query, sections["abstract"][0])

CPU times: total: 15.6 ms
Wall time: 1.15 s


'The licenses mentioned in the context are the Apache 2.0 license.'

In [18]:
%%time

query = 'Is Jacob Austin an author of this paper?'
print(query)
answer = query_engine.query(query)
print(answer.response)
find = re.findall(r"'page_label': '[^']*'", str(answer.metadata))
page_num = eval("{" + find[0] + "}")["page_label"]

Is Jacob Austin an author of this paper?
Yes.
CPU times: total: 31.2 ms
Wall time: 1.42 s


In [19]:
%%time

ask(query, sections["authors"][0])

CPU times: total: 31.2 ms
Wall time: 1.11 s


'No, Jacob Austin is not mentioned as an author of this paper.'